In [1]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge librosa ffmpeg

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# IPython.display for audio output
import IPython.display as ipd
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model, Sequential
import keras.backend as K
import tensorflow as tf

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display


Bad key "text.kerning_factor" on line 4 in
/root/anaconda3/envs/ten2.1/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
Using TensorFlow backend.


In [89]:
audio_path = '/root/datasets/ai_challenge/SoundIdea/SoundIdea/6000/'
generate_path = './generated_noise/'


dir = np.array(os.listdir(audio_path))
train_label = [i for i,j in np.char.split(dir, sep=' ')]
label_list = sorted(list(set(train_label)))
for i in range(len(train_label)):
    train_label[i] = label_list.index(train_label[i])
resample_sr = 16000

feature = 'mels'
length = 3 # seconds
noise_dim = 300
if not( feature in ('stft', 'mels')):
    raise ValueError('wrong feature')
class_num = len(label_list)
BATCH_SIZE = 32
EPOCHS = 100

In [ ]:
resampled_data, sr = [], []
import time
start_time = time.time()

step = 0
for k,i in enumerate(dir):
    _audio_data, _sr = librosa.load(audio_path+i, sr=None)
    if _audio_data.shape[0] / _sr < length:
        t = (length * _sr - _audio_data.shape[0])
        _audio_data = np.pad(_audio_data, (t//2, t - (t//2)), 'constant', constant_values=(0))
        if _audio_data.shape[0] != length * _sr:
            raise ValueError('something wrong with doing pad')
    elif _audio_data.shape[0] / _sr > length:
        _audio_data = _audio_data[:length * _sr]
    resampled_data.append(librosa.resample(_audio_data, _sr, resample_sr) if _sr != resample_sr else _audio_data)
    sr.append(_sr)
    if step % 100 == 0:
        print(f'{step}/{dir.shape[0]}')
    step += 1
print(f'complete {time.time() - start_time} seconds')
resampled_data = np.array(resampled_data)

0/7546
100/7546
200/7546
300/7546
400/7546
500/7546
600/7546
700/7546
800/7546
900/7546
1000/7546
1100/7546
1200/7546
1300/7546
1400/7546
1500/7546
1600/7546
1700/7546
1800/7546
1900/7546
2000/7546
2100/7546
2200/7546
2300/7546
2400/7546


In [ ]:
shape = None
if feature == 'stft':
    shape = librosa.stft(resampled_data[0]).shape
elif feature == 'mels':
    shape = librosa.feature.melspectrogram(resampled_data[0], sr=resample_sr, n_mels=128).shape
else:
    raise ValueError('wrong feature')
shape

In [ ]:
ipd.Audio(audio_path+dir[0])

In [ ]:
def build_generator(output_shape=shape, class_num=class_num, stddev=0.2, z_dim=noise_dim):
    model = Sequential()

    model.add(Dense(256, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(output_shape), activation='tanh'))
    model.add(Reshape(output_shape))

    noise = Input(shape=(z_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(class_num, z_dim)(label))

    model_input = multiply([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)

    



def build_discriminator(input_shape=shape, class_num=class_num, stddev=0.2):
    noise_input = Input(shape=input_shape)
    reshaped_noise = Flatten()(noise_input)

    model = Sequential()

    model.add(Dense(512, input_dim=np.prod(input_shape)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=input_shape)
    label = Input(shape=(1,), dtype='int32')

    label_embedding = Flatten()(Embedding(class_num, np.prod(input_shape))(label))
    flat_img = Flatten()(img)

    model_input = multiply([flat_img, label_embedding])

    validity = model(model_input)

    return Model([img, label], validity)

  
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        d_loss_real = [0,0]
        d_loss_fake = [0,0]
        g_loss = 0
        step = 0

        for noise_batch, label_batch in dataset:
#             noise_batch = tf.cast(noise_batch, tf.float32)

            noise = np.random.normal(0, 1, (noise_batch.shape[0], noise_dim))
            # labels = tf.one_hot(labels,10)
            label_batch = tf.reshape(label_batch,(noise_batch.shape[0], -1))
            generated_noise = generator.predict([noise,label_batch])

            valid = np.ones((noise_batch.shape[0], 1))
            fake = np.zeros((noise_batch.shape[0], 1))

            d_loss_real = np.add(d_loss_real, discriminator.train_on_batch([noise_batch, label_batch], valid))
            d_loss_fake = np.add(d_loss_fake, discriminator.train_on_batch([generated_noise, label_batch], fake))
            sampled_labels = np.random.randint(0, class_num, noise_batch.shape[0])
            g_loss = np.add(g_loss, combined.train_on_batch([noise, sampled_labels], valid))
            step += 1
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0]/step, 100*d_loss[1]/step, g_loss/step))


        # GIF를 위한 이미지를 바로 생성합니다.
        # generate_and_save_noises(generator,
        #                             epoch + 1,
        #                             seed)
        sample_noises(generator, epoch+1)

        # 5 에포크가 지날 때마다 모델을 저장합니다.
        if (epoch + 1) % 5 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        # print (' 에포크 {} 에서 걸린 시간은 {} 초 입니다'.format(epoch +1, time.time()-start))
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    sample_noises(generator, epochs)
    

def preprocessing(data):
    train_data = []
    if feature == 'stft':
        for i in range(len(data)):
            train_data.append(librosa.stft(data[i]))
    elif feature == 'mels':
        for i in range(len(data)):
            print('hi')
            train_data.append(librosa.feature.melspectrogram(data[i], sr=resample_sr, n_mels=128))
    return np.array(train_data)


In [ ]:
def sample_noises(generator, epoch):
    r, c = 2, 5
    noise = np.random.normal(0, 1, (r * c, noise_dim))
    sampled_labels = np.arange(0, 10).reshape(-1, 1)

    gen_sound = generator.predict([noise, sampled_labels])
    sampled_labels = np.arange(0, 10).reshape(-1)
    for i, j in enumerate(sampled_labels):
        librosa.output.write_wav(os.path.join(generate_path, f'{epoch}_{label_list[j]}.wav'), librosa.istft(gen_sound[i]), resample_sr, norm=True)


In [ ]:


train_data = preprocessing(resampled_data)

In [79]:
generator = build_generator()
discriminator = build_discriminator()

generator_optimizer = tf.keras.optimizers.Adam(0.001, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0001, 0.5)
discriminator.compile(loss=['binary_crossentropy'],
        optimizer=discriminator_optimizer,
        metrics=['acc'])
noise = Input(shape=(noise_dim,))
label = Input(shape=(1,))
sound = generator([noise, label])
discriminator.trainable = False
valid = discriminator([sound, label])
combined = Model([noise, label], valid)
combined.compile(loss=['binary_crossentropy'],
        optimizer=generator_optimizer)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)

In [80]:
perm = np.random.permutation(len(train_data))
train_noises = np.array(train_data)[perm]
train_labels = np.array(train_label)[perm]

train_dataset = tf.data.Dataset.from_tensor_slices((train_noises, train_labels)).batch(BATCH_SIZE)
train(train_dataset, EPOCHS)

/root/anaconda3/envs/ten2.1/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in double_scalars


ZeroDivisionError: division by zero

In [ ]:
sample_noises(generator, 16)

In [ ]:
generator.summary()

In [66]:
path = './generated_noise'
sorted(os.listdir(path))

['10_AddingMachine',
 '10_AddingMachine.wav',
 '10_AirConditioner',
 '10_AirConditioner.wav',
 '10_AirRegulator',
 '10_AirRegulator.wav',
 '10_AirRelease',
 '10_AirRelease.wav',
 '10_AircraftCarrier',
 '10_AircraftCarrier.wav',
 '10_AirplaneBiplane',
 '10_AirplaneBiplane.wav',
 '10_AirplaneCabin',
 '10_AirplaneCabin.wav',
 '10_AirplaneCockpit',
 '10_AirplaneCockpit.wav',
 '10_AirplaneJet',
 '10_AirplaneJet.wav',
 '10_AirplaneSingleProp',
 '10_AirplaneSingleProp.wav',
 '11_AddingMachine',
 '11_AddingMachine.wav',
 '11_AirConditioner',
 '11_AirConditioner.wav',
 '11_AirRegulator',
 '11_AirRegulator.wav',
 '11_AirRelease',
 '11_AirRelease.wav',
 '11_AircraftCarrier',
 '11_AircraftCarrier.wav',
 '11_AirplaneBiplane',
 '11_AirplaneBiplane.wav',
 '11_AirplaneCabin',
 '11_AirplaneCabin.wav',
 '11_AirplaneCockpit',
 '11_AirplaneCockpit.wav',
 '11_AirplaneJet',
 '11_AirplaneJet.wav',
 '11_AirplaneSingleProp',
 '11_AirplaneSingleProp.wav',
 '12_AddingMachine',
 '12_AddingMachine.wav',
 '12_AirCo

In [68]:
ipd.Audio(generate_path+'5_AirRelease.wav')